In [10]:
train=False
test=True

if test:
    import gc
    import numpy as np
    from keras.preprocessing import image
    from PIL import Image
    from keras.utils import np_utils
    from keras.models import model_from_json
    from keras.models import load_model

if train:
    import gc
    import numpy as np
    from keras.datasets import cifar10
    from keras.models import Sequential
    from keras.layers import Dense, Flatten, Activation
    from keras.layers import Dropout
    from keras.layers.convolutional import Conv2D, MaxPooling2D
    from keras.utils import np_utils
    from keras.optimizers import SGD

# Задаем seed для повторяемости результатов
    np.random.seed(42)

# Загружаем данные
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Размер мини-выборки
    batch_size = 32
# Количество классов изображений
    nb_classes = 10
# Количество эпох для обучения
    nb_epoch = 30
# Размер изображений
    img_rows, img_cols = 32, 32

# Нормализуем данные
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255

# Преобразуем метки в категории
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)

# Создаем последовательную модель
    model = Sequential()
# Первый сверточный слой
    model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(32, 32, 3), activation='relu'))
# Второй сверточный слой
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Первый слой подвыборки
    model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
    model.add(Dropout(0.25))

# Третий сверточный слой
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Четвертый сверточный слой
    model.add(Conv2D(64, (3, 3), activation='relu'))
# Второй слой подвыборки
    model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
    model.add(Dropout(0.25))
# Слой преобразования данных из 2D представления в плоское
    model.add(Flatten())
# Полносвязный слой для классификации
    model.add(Dense(512, activation='relu'))
# Слой регуляризации Dropout
    model.add(Dropout(0.5))
# Выходной полносвязный слой
    model.add(Dense(nb_classes, activation='softmax'))

# Задаем параметры оптимизации
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# Обучаем модель
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)

# Оцениваем качество обучения модели на тестовых данных
    scores = model.evaluate(X_test, Y_test, verbose=0)
    print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))
    model.save('my_model.h5')


if test:
    classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']
    if train==False:
#         json_file = open("cifar10_model.json", "r")
#         loaded_model_json = json_file.read()
#         json_file.close()
#         loaded_model = model_from_json(loaded_model_json)
        loaded_model=load_model("my_model.h5")
        loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    i=1
    while i<11:
        img_path = str(i)+'.jpg'
        img = image.load_img(img_path, target_size=(32, 32))
        x = image.img_to_array(img)
        x /= 255
        x = np.expand_dims(x, axis=0)

        if train==False:
            prediction = loaded_model.predict(x)
        else:
            prediction = model.predict(x)

        print(str(i)+" "+classes[np.argmax(prediction)])
        i=i+1
        
    scores = loaded_model.evaluate(X_test, Y_test, verbose=0)
    print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

1 автомобиль
2 автомобиль
3 автомобиль
4 автомобиль
5 автомобиль
6 грузовик
7 автомобиль
8 автомобиль
9 грузовик
10 автомобиль
Точность работы на тестовых данных: 76.58%
